In [1]:
#hide
#default_exp utils.experiment_utils
from nbdev.showdoc import *
from dsblocks.utils.nbdev_utils import nbdev_setup, TestRunner

nbdev_setup ()
tst = TestRunner (targets=['dummy'])
#tst = TestRunner (targets=[])

# Experiment Utils

> Helper functions for querying and retrieving results from past experiments

In [2]:
#export
import pandas as pd
import numpy as np
import pickle
import os
import sys
import time
from sklearn.model_selection import ParameterGrid
import warnings

from hpsearch.config import hp_defaults as dflt
warnings.filterwarnings('ignore')

In [3]:
#for tests
import pytest
import pandas as pd
from IPython.display import display

from dsblocks.utils.nbdev_utils import md
from hpsearch.examples.dummy_experiment_manager import (DummyExperimentManager, 
                                                        run_multiple_experiments)
from hpsearch.examples.complex_dummy_experiment_manager import generate_data, init_em
from hpsearch.config import hp_defaults as dflt

In [4]:
# exports tests.utils.test_experiment_utils
def generate_data_exp_utils (name_folder):
    path_experiments = f'test_{name_folder}/debug'
    manager_path = f'{path_experiments}/managers'
    em = DummyExperimentManager (path_experiments=path_experiments, manager_path=manager_path,
                                 verbose=0)
    em.remove_previous_experiments (parent=True)
    run_multiple_experiments(em=em, nruns=5, noise=0.1, verbose=False,
                             values_to_explore=dict(offset=[0.1, 0.3, 0.6], epochs=[5, 10, 100]))
    run_multiple_experiments(em=em, nruns=5, noise=0.1, verbose=False, rate=0.0001,
                             values_to_explore=dict(offset=[0.1, 0.3, 0.6], epochs=[5, 10, 100]))
    return em

## get_experiment_data

In [5]:
#export
def get_experiment_data (path_experiments=None, folder_experiments=None, experiments=None):
    """
    Returns data stored from previous experiments in the form DataFrame. 
    
    If path_experiments is not given, it uses the default one. 
    """
    from hpsearch.config.hpconfig import get_experiment_data
    return get_experiment_data (experiments=experiments)

### Usage example

In [6]:
#exports tests.utils.test_experiment_utils
def test_get_experiment_data ():
    path_experiments = 'get_experiment_data'
    em = generate_data (path_experiments)
    
    df = get_experiment_data ()
    reference = em.get_experiment_data ()
    pd.testing.assert_frame_equal (df, reference)
    
    em.remove_previous_experiments (parent=True)

In [7]:
tst.run (test_get_experiment_data, tag='dummy')

running test_get_experiment_data


NameError: name 'dflt' is not defined

## get experiment parameters and scores

### get_parameters_columns

In [8]:
#export
def get_parameters_columns (experiment_data, only_not_null=False):
    parameters = experiment_data[dflt.parameters_col].columns
    parameters = [(dflt.parameters_col, *x) for x in parameters]
    if only_not_null:
        parameters = np.array(parameters)[~experiment_data.loc[:,parameters].isnull().all(axis=0)].tolist()
        parameters = [(*x,) for x in parameters]
    return parameters

### get_experiment_parameters

In [ ]:
#export
def get_experiment_parameters (experiment_data, only_not_null=False):
    return experiment_data[get_parameters_columns (experiment_data, only_not_null=only_not_null)]

### get_scores_columns

In [ ]:
#export
def get_scores_columns (experiment_data=None, score_name=None, run_number = None):
    """
    Determine the columnns that provide evaluation scores.
    """
    if run_number is not None:
        scores_columns = experiment_data[dflt.scores_col].columns.get_level_values(0)
        scores_columns = [(dflt.scores_col, x) for x in scores_columns]
    else:
        if experiment_data is None:
            raise ValueError ('Either experiment_data or run_number should be different than None')
        if score_name is not None:
            scores_columns = (dflt.scores_col, score_name)
        else:
            scores_columns = experiment_data[dflt.scores_col].columns
            scores_columns = [(dflt.scores_col, *x) for x in scores_columns]
    return scores_columns

### get_experiment_scores

In [ ]:
#export
def get_experiment_scores (experiment_data = None, score_name=None, run_number=None, remove_score_name=False):
    df = experiment_data[get_scores_columns (experiment_data, score_name=score_name, run_number=run_number)]
    return df

### Usage examples

In [ ]:
#exports tests.utils.test_experiment_utils
def test_get_parameters_and_scores ():
    path_experiments = 'test_get_parameters_and_scores'
    em = generate_data (path_experiments)
    df = em.get_experiment_data ()

    # ************************************************************
    # get_parameters_columns
    # ************************************************************
    expected_result = [(dflt.parameters_col, x, '') for x in ['epochs', 'noise', 'offset', 'rate']]
    assert get_parameters_columns (df) == expected_result

    mi_offset = (dflt.parameters_col, 'offset', '')
    offset = df[mi_offset].values.copy()
    md ('- We can take only those which have at least some value that is not None.')
    df.loc[:, mi_offset] = None
    expected_result = [(dflt.parameters_col, x, '') for x in ['epochs', 'noise', 'rate']]
    assert get_parameters_columns (df, only_not_null=True) == expected_result

    md ('- If only some elements are None for a given parameter, we still include it.')
    df.loc[:, mi_offset] = offset
    df.loc[2, mi_offset] = None
    expected_result = [(dflt.parameters_col, x, '') for x in ['epochs', 'noise', 'offset', 'rate']]
    assert get_parameters_columns (df, only_not_null=True)==expected_result
    df.loc[:, mi_offset] = offset

    # ************************************************************
    # get_experiment_parameters
    # ************************************************************
    md ('- Same as get_parameters_columns, but returning dataframe of parameter values.')
    result = get_experiment_parameters (df)
    assert result.shape == (9, 4)
    expected_result = [(dflt.parameters_col, x, '') for x in ['epochs', 'noise', 'offset', 'rate']]
    assert result.columns.tolist() == expected_result

    # ************************************************************
    # get_scores_columns
    # ************************************************************
    md ('- Retrieve all columns that have scores, for all runs')
    expected_result = [(dflt.scores_col, x, y) for x in ['validation_accuracy', 'test_accuracy'] 
                       for y in range(5)]
    assert get_scores_columns (df) == expected_result

    md ('- Retrieve all columns for given score name, for all runs')
    expected_result = [(dflt.scores_col, x, y) for x in ['test_accuracy'] 
                       for y in range(5)]
    assert get_scores_columns (df, score_name='test_accuracy') == expected_result

    md ('- Retrieve all columns for given score name, for given runs')
    expected_result = [(dflt.scores_col, x, y) for x in ['test_accuracy'] 
                       for y in [2, 4]]
    assert get_scores_columns (df, score_name='test_accuracy', run_number=[2, 4]) == expected_result

    # ************************************************************
    # get_experiment_scores
    # ************************************************************
    md ('- Same, but returning dataframe with selected scores values:')
    result = get_experiment_scores (df)
    display (result)
    assert result.shape==(9,15)

    result = get_experiment_scores (df, score_name='_test_accuracy')
    display (result)
    assert result.shape==(9,5)

    result = get_experiment_scores (df, score_name='_test_accuracy', run_number=[2,4])
    display (result)
    assert result.shape==(9,2)

    md ('- We can remove the metric name and only keep the run number in each column:')
    result = get_experiment_scores (df, score_name='_test_accuracy', run_number=[2,4], remove_suffix=True)
    display (result)
    assert result.shape==(9,2)
    
    # ************************************************************
    # get_scores_columns, first usage example: we do not indicate the name of the score
    # ************************************************************
    expected_result = [(dflt.scores_col, x, y) for x in ['validation_accuracy', 'test_accuracy'] 
                       for y in range(5)]
    assert get_scores_columns (df)==expected_result
    
    # ************************************************************
    # get_scores_columns, second usage: we indicate the name of the score
    # ************************************************************
    result = get_scores_columns (df, run_number=range(5), score_name='validation_accuracy')
    expected_result = [(dflt.scores_col, 'validation_accuracy', y) for y in range(5)]
    assert result == expected_result
    em.remove_previous_experiments (parent=True)

In [ ]:
tst.run (test_get_parameters_and_scores, tag='dummy')

## get_scores_names

In [ ]:
#export
def get_scores_names (experiment_data=None, run_number=None, experiment=None, only_valid=True):
    """ 
    Determine the names of the scores included in experiment data. 
    
    We assume that the score columns start with the class number, and that the other columns do not start with a digit.

    If run_number is provided, we provide the scores stored for that run number. If, in addition to this, 
    experiment is provided, and only_valid=True, we provide only the scores that are not NaN for the given 
    experiment number.
    """
    
    if run_number is None:
        scores_names = np.unique([('_'.join(col.split('_')[1:]) if (len(col.split('_')) > 1) else '') 
                                    for col in experiment_data.columns if col[0].isdigit()])
        
    else:
        scores_names = [col.split(f'{run_number}')[1] for col in experiment_data.columns if col.startswith(str(run_number))]
        scores_names = [('_'.join(col.split('_')[1:]) if (len(col.split('_')) > 1) else '')
                                    for col in scores_names]
        if (experiment is not None) and only_valid:
            scores_names = [name for name in scores_names if not np.isnan(experiment_data.loc[experiment, f'{run_number}_{name}'])]
        scores_names = list(np.sort(scores_names))
    # remove special names
    scores_names = [name for name in scores_names if name != 'finished']
    return scores_names

### Usage

In [ ]:
#exports tests.utils.test_experiment_utils
def test_get_scores_names ():
    em = generate_data_exp_utils ('get_scores_names')
    
    df = em.get_experiment_data ()
    scores_names = get_scores_names (df)
    print (scores_names)
    assert scores_names == ['test_accuracy', 'validation_accuracy']
    
    scores_names=get_scores_names (df, run_number=3, experiment=7)
    print(scores_names)
    assert list(np.sort(scores_names))==['test_accuracy', 'validation_accuracy']

    # test when only some scores are valid
    df2 = df.copy()
    df2.loc[7, '3_test_accuracy']=np.nan
    scores_names=get_scores_names (df2, run_number=3, experiment=7)
    print (scores_names)
    assert scores_names==['validation_accuracy']
    
    em.remove_previous_experiments (parent=True)

In [ ]:
tst.run (test_get_scores_names, tag='dummy')

## get_monitored_training_metrics

In [ ]:
#export
def get_monitored_training_metrics (experiment, run_number=0, history_file_name='model_history.pk', 
                                    path_results=None):
    if path_results is None:
        from hpsearch.config.hpconfig import get_path_results
        path_results = get_path_results(experiment, run_number)
    path_history = f'{path_results}/{history_file_name}'
    if os.path.exists(path_history):
        history=pickle.load(open(path_history,'rb'))
        return list(history.keys())
    else:
        return []

### Usage

In [ ]:
#exports tests.utils.test_experiment_utils
def test_get_monitored_training_metrics ():
    em = generate_data_exp_utils ('get_monitored_training_metrics')
    
    monitored_metrics = get_monitored_training_metrics (0)
    print (monitored_metrics)
    assert monitored_metrics==['validation_accuracy', 'test_accuracy', 'accuracy']
    
    em.remove_previous_experiments (parent=True)

In [ ]:
tst.run (test_get_monitored_training_metrics, tag='dummy')

## run_number

In [ ]:
#export
def get_runs_with_results (experiment_data = None, score_name=None, run_number=None):
    """
    Gets the list of run_number for whom there are results in experiment_data.
    """
    assert experiment_data is not None, 'experiment_data must be introduced'
    result_columns = get_scores_columns (experiment_data, score_name=score_name, run_number=run_number)
    completed_results = ~experiment_data.loc[:,result_columns].isnull()
    completed_results = completed_results.all(axis=0)
    completed_results = completed_results.iloc[np.where(completed_results)]
    completed_results = completed_results.index

    #return [int(x[:-len(score_name)]) for x in completed_results]

### Usage

In [ ]:
#exports tests.utils.test_experiment_utils
def test_get_runs_with_results ():
    em = generate_data ('get_runs_with_results')
    
    df = em.get_experiment_data ()
    # we need to introduce experiment_data df, and score_name
    result = get_runs_with_results (df, score_name='validation_accuracy')
    display (result)
    assert result==[0,1,2,3,4]
    
    # we can also restrict to certain run_number
    result = get_runs_with_results (df, score_name='validation_accuracy', run_number=[0,2])
    display (result)
    assert result==[0,2]
    em.remove_previous_experiments (parent=True)

In [ ]:
tst.run (test_get_runs_with_results, tag='dummy')

## get_parameters_unique

In [ ]:
#export
def get_parameters_unique(df):
    parameters = []
    for k in df.columns:
        if len(df[k].unique()) > 1:
            parameters += [k]
    return parameters, df[parameters]

### Usage

In [ ]:
#exports tests.utils.test_experiment_utils
def test_get_parameters_unique ():
    em = generate_data_exp_utils ('get_parameters_unique')
    df = em.get_experiment_data ()
    
    # keeps only those parameters with more than one value,
    # removing 'noise' in this case, since it has the same value in all rows
    result = get_parameters_unique (df[['epochs','offset','rate', 'noise']])
    assert result[1].shape==(18,3)
    assert result[0] == ['epochs', 'offset', 'rate']
    
    em.remove_previous_experiments (parent=True)

In [ ]:
tst.run (test_get_parameters_unique, tag='dummy')

## compact_parameters

In [ ]:
#export
def compact_parameters (df, number_characters=1):
    par_or = df.columns
    par_new = [''.join(y[0].upper()+y[1:number_characters] for y in x.split('_')) for x in par_or]
    dict_rename = {k:v for k,v in zip(par_or, par_new)}
    df = df.rename (columns = dict_rename)
    
    return df, dict_rename

### Usage

In [ ]:
#exports tests.utils.test_experiment_utils
def test_compact_parameters ():
    em = generate_data_exp_utils ('compact_parameters')
    df = em.get_experiment_data ()
    
    result = compact_parameters (df, number_characters=2)
    display (result[0].head())
    assert all(result[0].columns == ['Ep', 'Of', 'Ra', 'No', '0VaAc', '0TeAc', 'Ti0', 'Da', '0Fi', '1VaAc',
           '1TeAc', 'Ti1', '1Fi', '2VaAc', '2TeAc', 'Ti2', '2Fi', '3VaAc', '3TeAc',
           'Ti3', '3Fi', '4VaAc', '4TeAc', 'Ti4', '4Fi'])

    assert result[1]=={'epochs': 'Ep',
         'offset': 'Of',
         'rate': 'Ra',
         'noise': 'No',
         '0_validation_accuracy': '0VaAc',
         '0_test_accuracy': '0TeAc',
         'time_0': 'Ti0',
         'date': 'Da',
         '0_finished': '0Fi',
         '1_validation_accuracy': '1VaAc',
         '1_test_accuracy': '1TeAc',
         'time_1': 'Ti1',
         '1_finished': '1Fi',
         '2_validation_accuracy': '2VaAc',
         '2_test_accuracy': '2TeAc',
         'time_2': 'Ti2',
         '2_finished': '2Fi',
         '3_validation_accuracy': '3VaAc',
         '3_test_accuracy': '3TeAc',
         'time_3': 'Ti3',
         '3_finished': '3Fi',
         '4_validation_accuracy': '4VaAc',
         '4_test_accuracy': '4TeAc',
         'time_4': 'Ti4',
         '4_finished': '4Fi'}
    
    em.remove_previous_experiments (parent=True)

In [ ]:
tst.run (test_compact_parameters, tag='dummy')

## replace_with_default_values

In [ ]:
#export
def replace_with_default_values (df, parameters={}):
    from hpsearch.config.hpconfig import get_default_parameters
    
    parameters_names = get_parameters_columns (df)
    
    for k in df.columns:
        experiments_idx=np.argwhere(df[k].isna().ravel()).ravel()
        experiments=df.index[experiments_idx]
        for experiment in experiments:
            parameters = df.loc[experiment, parameters_names].copy()
            parameters[parameters.isna().values] = None
            parameters = parameters.to_dict()
            parameters = {k:parameters[k] for k in parameters if parameters[k] is not None}
            defaults = get_default_parameters(parameters)
            df.loc[experiment, k] = defaults.get(k)
    return df

### Usage

In [ ]:
#exports tests.utils.test_experiment_utils
def test_replace_with_default_values ():
    em = generate_data_exp_utils ('replace_with_default_values')
    
    df = em.get_experiment_data ()
    df=replace_with_default_values(df)
    mi_epoch = (dflt.parameters_col, 'epochs', '')
    assert (df[mi_epoch].values == ([5.]*3 + [10.]*3 + [100.]*3)*2).all()
    
    em.remove_previous_experiments (parent=True)

In [ ]:
tst.run (test_replace_with_default_values, tag='dummy')

## remove_defaults

In [ ]:
#export
def remove_defaults (parameters):
    from hpsearch.config.hpconfig import get_default_parameters
    
    defaults = get_default_parameters(parameters)
    for key in defaults.keys():
        if key in parameters.keys() and (parameters[key] == defaults[key]):
            del parameters[key]
    return parameters

### Usage example

In [ ]:
#exports tests.utils.test_experiment_utils
def test_remove_defaults ():
    em = init_em ('remove_defaults')
    result, dict_results = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05})
    
    parameters = remove_defaults ({'offset':0.1, 'rate': 0.05})
    assert parameters=={'offset':0.1, 'rate': 0.05}
    
    parameters = remove_defaults ({'offset':0.1, 'rate': 0.01, 'epochs': 10})
    assert parameters=={'offset':0.1}
    
    parameters = remove_defaults ({'offset':0.5, 'rate': 0.000001, 'epochs': 10})
    assert parameters=={'rate': 0.000001, 'epochs': 10}
    
    parameters = remove_defaults ({'offset':0.5, 'rate': 0.000001, 'epochs': 100})
    assert parameters=={'rate': 0.000001}
    
    em.remove_previous_experiments (parent=True)

In [ ]:
tst.run (test_remove_defaults, tag='dummy')

## find_rows_with_parameters_dict

In [35]:
#export
def find_rows_with_parameters_dict (experiment_data, parameters_dict, create_if_not_exists=True, 
                                    exact_match=True, ignore_keys=[], precision = 1e-10):
    """
    Finds rows that match parameters. 
    
    If the dataframe doesn't have any parameter with that name, a new column 
    is created and changed_dataframe is set to True.
    """
    changed_dataframe = False
    matching_all_condition = pd.Series([True]*experiment_data.shape[0])
    existing_keys = [par for par in parameters_dict.keys() if par not in ignore_keys]
    for parameter in existing_keys:
        mi_parameter = (dflt.parameters_col, parameter, '')
        if mi_parameter not in experiment_data.columns:
            if create_if_not_exists:
                experiment_data[mi_parameter] = None
                changed_dataframe = True
            else:
                raise ValueError ('parameter %s not found in experiment_data' %parameter)
        if parameters_dict[parameter] is None:
            matching_condition = experiment_data[mi_parameter].isnull()
        elif experiment_data[mi_parameter].isnull().all():
            matching_condition = ~experiment_data[mi_parameter].isnull()
        elif (type(parameters_dict[parameter]) == float) or (type(parameters_dict[parameter]) == np.float32) or (type(parameters_dict[parameter]) == np.float64):
            if parameters_dict[parameter] == np.floor(parameters_dict[parameter]):
                matching_condition = experiment_data[mi_parameter]==parameters_dict[parameter]
            else:
                matching_condition = experiment_data[mi_parameter]==parameters_dict[parameter]
                for idx, v in enumerate(experiment_data[mi_parameter]):
                    if (type(v) == float or type(v) == np.float32 or type(v) == np.float64) and (np.abs(v-parameters_dict[parameter]) < precision):
                        matching_condition.iloc[idx]=True
                    else:
                        matching_condition.iloc[idx]=False
        else:
            matching_condition = experiment_data[mi_parameter]==parameters_dict[parameter]

        matching_all_condition = matching_all_condition & matching_condition.values
            
    # We assume that all the columns correspond to parameters, except for those that start with a digit (corresponding to the class evaluated) and those that start with time (giving an estimation of the computational cost)
    if exact_match:
        rest_parameters = experiment_data[dflt.parameters_col].columns.get_level_values(0)
        rest_parameters = [par for par in rest_parameters if par not in parameters_dict.keys()]
        rest_parameters = [par for par in rest_parameters if par not in ignore_keys]
        for parameter in rest_parameters:
            mi_parameter = (dflt.parameters_col, parameter, '')
            matching_condition = experiment_data[mi_parameter].isnull()
            matching_all_condition = matching_all_condition & matching_condition.values
    
    matching_rows = matching_all_condition.index[matching_all_condition].tolist()
    
    return matching_rows, changed_dataframe, matching_all_condition

### Usage example

In [36]:
#exports tests.utils.test_experiment_utils
def test_find_rows_with_parameters_dict ():
    em = generate_data_exp_utils ('find_rows_with_parameters_dict')
    
    df = em.get_experiment_data ()
    result = find_rows_with_parameters_dict (df, dict (rate=0.0001))
    matching_rows, changed_dataframe, matching_all_condition = result
    assert matching_rows==[]
    assert not changed_dataframe
    
    result = find_rows_with_parameters_dict (df, dict (rate=0.0001), exact_match=False)
    matching_rows, changed_dataframe, matching_all_condition = result
    assert matching_rows == [9, 10, 11, 12, 13, 14, 15, 16, 17]
    
    result = find_rows_with_parameters_dict (df, dict (rate=0.0001, epochs=5, offset=0.6), exact_match=False,
                                        ignore_keys=['epochs'])
    matching_rows, changed_dataframe, matching_all_condition = result
    assert matching_rows==[11, 14, 17]

    mi_rate = (dflt.parameters_col, 'rate', '')
    df.loc[16, mi_rate]=0.00011
    result = find_rows_with_parameters_dict (df, dict (rate=0.0001), exact_match=False)
    matching_rows, changed_dataframe, matching_all_condition = result
    assert matching_rows==[9, 10, 11, 12, 13, 14, 15, 17]

    result = find_rows_with_parameters_dict (df, dict (rate=0.0001), exact_match=False, precision = 0.0001)
    matching_rows, changed_dataframe, matching_all_condition = result
    assert matching_rows==[9, 10, 11, 12, 13, 14, 15, 16, 17]

    result = find_rows_with_parameters_dict (df, dict (new_par=4), exact_match=False)
    matching_rows, changed_dataframe, matching_all_condition = result
    assert changed_dataframe
    assert df.shape == (18, 26)
    assert matching_rows==[]
    assert 'new_par' in df.columns
    assert matching_rows==[]
    
    em.remove_previous_experiments (parent=True)

In [37]:
tst.run (test_find_rows_with_parameters_dict, tag='dummy')

running test_find_rows_with_parameters_dict


## summarize_results

In [38]:
#export
def summarize_results(path_experiments = None, 
                      folder_experiments = None,
                      intersection = False, 
                      experiments = None, 
                      score_name='', 
                      min_results=0, 
                      run_number = None, 
                      parameters = None,
                      output='all',
                      data = None,
                      ascending=False,
                      suffix_test_set = None,
                      stats = ['mean','median','rank','min','max','std']):
    """Obtains summary scores for the desired list of experiments. Uses the experiment_data csv for 
    that purpose
    
    Example use: 
        - restricting run_number:
            summarize_results(run_number= [1058,1059],score_name='_m3');
    
        - with a predetermined list of run_number:
            summarize_results(run_number='qualified',score_name='_m3',min_results=96);
    """
    
    
    if data is None:
        experiment_data = get_experiment_data (path_experiments=path_experiments, folder_experiments=folder_experiments)
        experiment_data_original = experiment_data.copy()
        if experiments is not None:
            experiment_data = experiment_data.loc[experiments,:]
        if parameters is not None:
            experiment_rows, _, _ = find_rows_with_parameters_dict (experiment_data, parameters, create_if_not_exists=False, exact_match=False)
            experiment_data = experiment_data.loc[experiment_rows]
    else:
        experiment_data = data.copy()
        experiment_data_original = experiment_data.copy()
        
    # Determine the columnns that provide evaluation scores. 
    result_columns = get_scores_columns (experiment_data, score_name=score_name, run_number=run_number)
    
    experiment_data.loc[:,'num_results'] = np.sum(~experiment_data.loc[:,result_columns].isnull(),axis=1)
    if min_results > 0:
        number_before = experiment_data.shape[0]
        experiment_data = experiment_data[experiment_data.num_results>=min_results]
        print (f'{experiment_data.shape[0]} out of {number_before} experiments have {min_results} runs completed')
    
    # Take only those run_number where all experiments provide some score
    if intersection:
        number_before = len(result_columns)
        all_have_results = ~experiment_data.loc[:,result_columns].isnull().any(axis=0)
        result_columns = (np.array(result_columns)[all_have_results]).tolist()
        print (f'{len(result_columns)} out of {number_before} runs for whom all the selected experiments have completed')
        
    print (f'total data examined: {experiment_data.shape[0]} experiments with at least {experiment_data["num_results"].min()} runs done for each one')
        
    scores = -experiment_data.loc[:,result_columns].values
    rank = np.argsort(scores,axis=0)
    rank = np.argsort(rank,axis=0).astype(np.float32)
    rank[experiment_data.loc[:,result_columns].isnull()]=np.nan
    
    parameters = get_parameters_columns(experiment_data, True)
    experiment_data.loc[:,'mean'] = experiment_data.loc[:,result_columns].mean(axis=1)
    experiment_data.loc[:,'min'] = experiment_data.loc[:,result_columns].min(axis=1)
    experiment_data.loc[:,'max'] = experiment_data.loc[:,result_columns].max(axis=1)
    experiment_data.loc[:,'std'] = experiment_data.loc[:,result_columns].std(axis=1)
    experiment_data.loc[:,'median'] = experiment_data.loc[:,result_columns].median(axis=1)
    experiment_data.loc[:,'rank'] = np.nanmean(rank,axis=1)
    experiment_data.loc[:,'good'] = (experiment_data.loc[:,result_columns]>=0.1666666).sum(axis=1)
    
    scores_to_return = dict(mean=['mean'], median=['median'], rank=['rank'], good=['good'])
    if suffix_test_set is not None:
        def add_score_to_return (suffix_test_set_i):
            result_columns_test_set = get_scores_columns (experiment_data, score_name=suffix_test_set_i, run_number=run_number)
            experiment_data.loc[:,'mean%s' %suffix_test_set_i] = experiment_data.loc[:,result_columns_test_set].mean(axis=1)
            experiment_data.loc[:,'median%s' %suffix_test_set_i] = experiment_data.loc[:,result_columns_test_set].median(axis=1)
            scores_test_set = -experiment_data.loc[:,result_columns_test_set].values
            rank_test_set = np.argsort(scores_test_set,axis=0)
            rank_test_set = np.argsort(rank_test_set,axis=0).astype(np.float32)
            rank_test_set[experiment_data.loc[:,result_columns].isnull()]=np.nan
            experiment_data.loc[:,'rank%s' %suffix_test_set_i] = np.nanmean(rank_test_set,axis=1)
            experiment_data.loc[:,'good%s' %suffix_test_set_i] = (experiment_data.loc[:,result_columns_test_set]>=0.1666666).sum(axis=1)
            for k in scores_to_return.keys():
                scores_to_return[k] += ['%s%s' %(k, suffix_test_set_i)]
        if type(suffix_test_set) == str:
            suffix_test_set = [suffix_test_set]
        for suffix_test_set_i in suffix_test_set:
            add_score_to_return(suffix_test_set_i)
        
    if output == 'all':
        summary = dict (mean = experiment_data.loc[:,parameters+scores_to_return['mean']].sort_values(by='mean',ascending=ascending),
                        median = experiment_data.loc[:,parameters+scores_to_return['median']].sort_values(by='median',ascending=ascending),
                        rank = experiment_data.loc[:,parameters+scores_to_return['rank']].sort_values(by='rank'),
                        good = experiment_data.loc[:,parameters+scores_to_return['good']].sort_values(by='good',ascending=False),
                        stats = experiment_data.loc[:,parameters+stats].sort_values(by='mean',ascending=ascending),
                        unordered = experiment_data.loc[:,parameters],
                        allcols = experiment_data,
                        original = experiment_data_original
                        )
    elif output == 'stats':
        summary = experiment_data.loc[:,parameters+['mean','median','rank']]
    elif output == 'unordered':
        summary = experiment_data.loc[:,parameters]
    elif output == 'allcols':
        summary = experiment_data
    elif output == 'original':
        summary = experiment_data_original
    else:
        summary = experiment_data.loc[:,parameters+[output]].sort_values(by=output, ascending=output=='rank')
        

    return summary

### Usage example

In [39]:
#exports tests.utils.test_experiment_utils
def test_summarize_results ():
    em = init_em ('summarize_results')
    em.run_multiple_repetitions (parameters=dict(offset=0.1, rate=0.01), nruns=3)
    em.run_multiple_repetitions (parameters=dict(offset=0.2, rate=0.001), nruns=5)
    em.run_multiple_repetitions (parameters=dict(offset=0.3, rate=0.02), nruns=2)
    
    md ('\n\n')
    md ('- We need to indicate the metric to be retrieved, otherwise it will count '
        'as many results as num_results*num_metrics: ')
    d = summarize_results ()
    display (d['mean'])
    assert d['mean'].num_results.sum() == 30
    assert d['mean'].shape[0]==3
    
    md ('\n\n')
    md ('- The metric is indicated with `_` at the beginning: ')
    d = summarize_results (score_name='validation_accuracy')
    display (d['mean'])
    assert d['mean'].num_results.sum() == 10
    assert d['mean'].shape[0]==3
    
    md ('\n\n')
    md ('- We can filter those results that have less than X runs: ')
    d = summarize_results (score_name='validation_accuracy', min_results=5)
    display (d['mean'])
    assert d['mean'].num_results.sum() == 5
    assert d['mean'].shape[0]==1
    
    md ('\n\n')
    md ('- We can filter by experiment number and/or number of results, and retrieve the original dataframe,'
        'plus new columns with stats: ')
    d = summarize_results (score_name='validation_accuracy', experiments=[0,2], output='allcols')
    display (d)
    assert d.shape[0]==2
    assert all(d.index==[0,2])
    assert {'mean', 'min', 'max', 'std', 'median'}.issubset(d.columns)
    em.remove_previous_experiments (parent=True)

In [40]:
tst.run (test_summarize_results, tag='dummy')

running test_summarize_results
fitting model with 10 epochs
epoch 0: accuracy: 0.11
epoch 1: accuracy: 0.12
epoch 2: accuracy: 0.13
epoch 3: accuracy: 0.14
epoch 4: accuracy: 0.15000000000000002
epoch 5: accuracy: 0.16000000000000003
epoch 6: accuracy: 0.17000000000000004
epoch 7: accuracy: 0.18000000000000005
epoch 8: accuracy: 0.19000000000000006
epoch 9: accuracy: 0.20000000000000007
fitting model with 10 epochs
epoch 0: accuracy: 0.11
epoch 1: accuracy: 0.12
epoch 2: accuracy: 0.13
epoch 3: accuracy: 0.14
epoch 4: accuracy: 0.15000000000000002
epoch 5: accuracy: 0.16000000000000003
epoch 6: accuracy: 0.17000000000000004
epoch 7: accuracy: 0.18000000000000005
epoch 8: accuracy: 0.19000000000000006
epoch 9: accuracy: 0.20000000000000007
fitting model with 10 epochs
epoch 0: accuracy: 0.11
epoch 1: accuracy: 0.12
epoch 2: accuracy: 0.13
epoch 3: accuracy: 0.14
epoch 4: accuracy: 0.15000000000000002
epoch 5: accuracy: 0.16000000000000003
epoch 6: accuracy: 0.17000000000000004
epoch 7: 

- We need to indicate the metric to be retrieved, otherwise it will count as many results as num_results*num_metrics: 

total data examined: 3 experiments with at least 6 runs done for each one


,offset,rate,num_results,mean
2,0.3,0.020,6,0.633333
1,0.2,0.001,15,0.440000
0,0.1,NaN,9,0.433333


- The metric is indicated with `_` at the beginning: 

...

1 out of 3 experiments have 5 runs completed
total data examined: 1 experiments with at least 5 runs done for each one


,offset,rate,num_results,mean
1,0.2,0.001,5,0.21


- We can filter by experiment number and/or number of results, and retrieve the original dataframe,plus new columns with stats: 

total data examined: 2 experiments with at least 2 runs done for each one


,offset,0_validation_accuracy,0_test_accuracy,time_0,date,0_finished,1_validation_accuracy,1_test_accuracy,time_1,1_finished,...,time_4,4_finished,num_results,mean,min,max,std,median,rank,good
0,0.1,0.2,0.1,0.002003,15:54:43.969089,True,0.2,0.1,0.001967,True,...,NaN,NaN,3,0.2,0.2,0.2,0.0,0.2,0.666667,3
2,0.3,0.5,0.4,0.002099,15:54:44.225272,True,0.5,0.4,0.002191,True,...,NaN,NaN,2,0.5,0.5,0.5,0.0,0.5,0.000000,2


## query

In [41]:
#export
def query (path_experiments = None, 
              folder_experiments = None,
              intersection = False, 
              experiments = None, 
              score_name='', 
              min_results=0, 
              run_number=None, 
              parameters_fixed = {},
              parameters_variable = {},
              parameters_all = [],
              exact_match = True,
              output='all',
              ascending=False,
              suffix_test_set = None,
              stats = ['mean','median','rank','min','max','std'],
              query_other_parameters=False):
  
    if path_experiments is None:
        from hpsearch.config.hpconfig import get_path_experiments
        path_experiments = get_path_experiments()
    
    path_pickle = None
    if query_other_parameters:
        path_csv = '%s/other_parameters.csv' %path_experiments
    else:
        path_pickle = '%s/experiments_data.pk' %path_experiments
        if not os.path.exists(path_pickle):
            path_pickle = None
            path_csv = '%s/experiments_data.csv' %path_experiments
    if path_pickle is not None:
        experiment_data = pd.read_pickle(path_pickle)
    else:
        experiment_data = pd.read_csv(path_csv, index_col=0)
    
    non_valid_pars = set(parameters_fixed.keys()).difference(set(experiment_data.columns))
    if len(non_valid_pars) > 0:
        print (f'\n**The following query parameters are not valid: {list(non_valid_pars)}**')
        print (f'\nValid parameters:\n{sorted(get_parameters_columns(experiment_data))}\n')
    
    parameters_multiple_values_all = list(ParameterGrid(parameters_variable))
    experiment_numbers = []
    for (i, parameters_multiple_values) in enumerate(parameters_multiple_values_all):
        parameters = parameters_multiple_values.copy()
        parameters.update(parameters_fixed)
        parameters_none = {k:v for k,v in parameters.items() if v is None}
        parameters_not_none = {k:v for k,v in parameters.items() if v is not None}

        parameters = remove_defaults (parameters_not_none)
        parameters.update(parameters_none)
    
        experiment_numbers_i, _, _ = find_rows_with_parameters_dict (experiment_data, parameters, ignore_keys=parameters_all, exact_match = exact_match)
        experiment_numbers += experiment_numbers_i
    
    experiment_data = experiment_data.iloc[experiment_numbers]
    
    if experiments is not None:
        experiment_data = experiment_data.loc[experiments]
        
    if query_other_parameters:
        return experiment_data
  
    d=summarize_results(path_experiments=path_experiments, 
                      folder_experiments=folder_experiments,
                      intersection=intersection, 
                      experiments=experiments, 
                      score_name=score_name, 
                      min_results=min_results, 
                      run_number=run_number, 
                      parameters=None,
                      output='all',
                      data=experiment_data,
                      ascending=ascending,
                      suffix_test_set=suffix_test_set,
                      stats=stats)
                      
    return d['mean'], d

### Usage example

In [48]:
#exports tests.utils.test_experiment_utils
def test_query ():
    em = generate_data_exp_utils ('query')
        
    dmean, d = query (parameters_fixed=dict (rate=0.0001))
    assert dmean.empty
    
    md ('the dataframe only has mean. Results are sorted by mean score')
    dmean, d = query (parameters_fixed=dict (rate=0.0001), exact_match=False)
    assert dmean.shape[0]==9 and (dmean.rate==0.0001).all() and len(dmean.offset.unique())==3 and dmean['mean'].iloc[0]>dmean['mean'].iloc[1] and dmean['mean'].iloc[1] > dmean['mean'].iloc[2]
    display (dmean)
    md ('The second output d contains a field "stats" which is a dataframe. Results are sorted by mean score')
    assert {'mean', 'median', 'rank', 'min', 'max', 'std'}.issubset(d['stats'])
    display (d['stats'])
    
    md ('We can request parameter be in specific list of values')   
    dmean, d = query (parameters_fixed=dict(rate=0.0001), exact_match=False, 
                  parameters_variable=dict(epochs=[5,10], offset=[0.1, 0.3]))
    assert sorted(dmean.epochs.unique()) == [5,10]
    assert sorted(dmean.offset.unique()) == [0.1, 0.3]
    assert dmean.shape[0]==4
    display (dmean)
    
    md ('If we want a value that is the default, we need to indicate None')
    dmean, d = query (parameters_fixed=dict(rate=0.0001), exact_match=False, 
              parameters_variable=dict(epochs=[10, None], offset=[0.1, 0.3]))
    assert dmean.shape[0]==4
    assert dmean.epochs.isna().sum() == 2
    assert (dmean.epochs == 10).sum() == 2
    display (dmean)
    
    em.remove_previous_experiments (parent=True)

In [49]:
tst.run (test_query, tag='dummy')

running test_query
total data examined: 0 experiments with at least nan runs done for each one


the dataframe only has mean. Results are sorted by mean score

total data examined: 9 experiments with at least 15 runs done for each one


,epochs,offset,rate,noise,num_results,mean
11,5.0,0.6,0.0001,0.1,15,0.731242
14,10.0,0.6,0.0001,0.1,15,0.693155
17,NaN,0.6,0.0001,0.1,15,0.679728
10,5.0,0.3,0.0001,0.1,15,0.548471
13,10.0,0.3,0.0001,0.1,15,0.505715
16,NaN,0.3,0.0001,0.1,15,0.481352
9,5.0,0.1,0.0001,0.1,15,0.432128
12,10.0,0.1,0.0001,0.1,15,0.386496
15,NaN,0.1,0.0001,0.1,15,0.380060


The second output d contains a field "stats" which is a dataframe. Results are sorted by mean score

...

total data examined: 4 experiments with at least 15 runs done for each one


,epochs,offset,rate,noise,num_results,mean
10,5.0,0.3,0.0001,0.1,15,0.548471
13,10.0,0.3,0.0001,0.1,15,0.505715
9,5.0,0.1,0.0001,0.1,15,0.432128
12,10.0,0.1,0.0001,0.1,15,0.386496


If we want a value that is the default, we need to indicate None

total data examined: 4 experiments with at least 15 runs done for each one


,epochs,offset,rate,noise,num_results,mean
13,10.0,0.3,0.0001,0.1,15,0.505715
16,NaN,0.3,0.0001,0.1,15,0.481352
12,10.0,0.1,0.0001,0.1,15,0.386496
15,NaN,0.1,0.0001,0.1,15,0.380060


## summary

In [50]:
#export
def summary (df, experiments = None, score=None, compact=True):
    if experiments is not None:
        df = df.loc[experiments]
    if compact:
        _, df = get_parameters_unique(df)
    parameters_columns = get_parameters_columns(df, True)
    scores_columns = get_scores_columns (df, score_name=score)
    df = df[parameters_columns + scores_columns]
    df.columns=[c.split('_')[0] for c in df.columns]
    return df

### Usage example

In [59]:
#exports tests.utils.test_experiment_utils
def test_summary ():
    em = init_em ('summary')
    em.run_multiple_repetitions (parameters=dict(offset=0.1, rate=0.01), nruns=3)
    em.run_multiple_repetitions (parameters=dict(offset=0.2, rate=0.001), nruns=5)
    em.run_multiple_repetitions (parameters=dict(offset=0.3, rate=0.02), nruns=2)
    df = em.get_experiment_data()
    result = summary (df, score='validation_accuracy')
    display (result)
    assert all(result.columns == ['offset', 'rate', '0', '1', '2', '3', '4'])
    assert result.shape[0] == 3
    em.remove_previous_experiments (parent=True)

In [60]:
tst.run (test_summary, tag='dummy')

running test_summary
fitting model with 10 epochs
epoch 0: accuracy: 0.11
epoch 1: accuracy: 0.12
epoch 2: accuracy: 0.13
epoch 3: accuracy: 0.14
epoch 4: accuracy: 0.15000000000000002
epoch 5: accuracy: 0.16000000000000003
epoch 6: accuracy: 0.17000000000000004
epoch 7: accuracy: 0.18000000000000005
epoch 8: accuracy: 0.19000000000000006
epoch 9: accuracy: 0.20000000000000007
fitting model with 10 epochs
epoch 0: accuracy: 0.11
epoch 1: accuracy: 0.12
epoch 2: accuracy: 0.13
epoch 3: accuracy: 0.14
epoch 4: accuracy: 0.15000000000000002
epoch 5: accuracy: 0.16000000000000003
epoch 6: accuracy: 0.17000000000000004
epoch 7: accuracy: 0.18000000000000005
epoch 8: accuracy: 0.19000000000000006
epoch 9: accuracy: 0.20000000000000007
fitting model with 10 epochs
epoch 0: accuracy: 0.11
epoch 1: accuracy: 0.12
epoch 2: accuracy: 0.13
epoch 3: accuracy: 0.14
epoch 4: accuracy: 0.15000000000000002
epoch 5: accuracy: 0.16000000000000003
epoch 6: accuracy: 0.17000000000000004
epoch 7: accuracy: 

,offset,rate,0,1,2,3,4
0,0.1,NaN,0.20,0.20,0.20,NaN,NaN
1,0.2,0.001,0.21,0.21,0.21,0.21,0.21
2,0.3,0.020,0.50,0.50,NaN,NaN,NaN
